In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Digital Alexandria: Artwork Authenticity Detection Model\n",
    "\n",
    "**Objective**: Build and train a machine learning model to detect artwork authenticity using computer vision and deep learning techniques.\n",
    "\n",
    "**Author**: William Couturier  \n",
    "**Date**: 2024\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import libraries\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.model_selection import train_test_split, cross_val_score\n",
    "from sklearn.preprocessing import StandardScaler, LabelEncoder\n",
    "from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier\n",
    "from sklearn.linear_model import LogisticRegression\n",
    "from sklearn.svm import SVC\n",
    "from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score\n",
    "from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score\n",
    "import tensorflow as tf\n",
    "from tensorflow import keras\n",
    "from tensorflow.keras import layers\n",
    "import cv2\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "print(\"🎨 Digital Alexandria - Authenticity Detection Model\")\n",
    "print(\"=\" * 55)\n",
    "print(f\"TensorFlow version: {tf.__version__}\")\n",
    "print(f\"GPU Available: {tf.config.list_physical_devices('GPU')}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Data Preparation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate synthetic artwork features dataset\n",
    "def generate_artwork_features(n_samples=5000):\n",
    "    \"\"\"\n",
    "    Generate synthetic artwork features for authenticity detection\n",
    "    Features include: color analysis, texture patterns, brush strokes, age indicators\n",
    "    \"\"\"\n",
    "    np.random.seed(42)\n",
    "    \n",
    "    # Authentic artworks (70% of dataset)\n",
    "    n_authentic = int(n_samples * 0.7)\n",
    "    \n",
    "    # Color features (RGB histogram statistics)\n",
    "    color_variance_auth = np.random.normal(0.15, 0.05, n_authentic)  # Authentic art has natural color variance\n",
    "    color_variance_fake = np.random.normal(0.25, 0.08, n_samples - n_authentic)  # Fakes often have irregular colors\n",
    "    \n",
    "    # Texture complexity (entropy-based measure)\n",
    "    texture_complexity_auth = np.random.normal(7.2, 1.1, n_authentic)  # Natural brush patterns\n",
    "    texture_complexity_fake = np.random.normal(5.8, 1.5, n_samples - n_authentic)  # Simpler or too complex\n",
    "    \n",
    "    # Brush stroke consistency\n",
    "    brush_consistency_auth = np.random.normal(0.8, 0.1, n_authentic)  # Consistent artist technique\n",
    "    brush_consistency_fake = np.random.normal(0.6, 0.15, n_samples - n_authentic)  # Inconsistent technique\n",
    "    \n",
    "    # Age indicators (cracking patterns, patina)\n",
    "    age_indicators_auth = np.random.normal(0.7, 0.2, n_authentic)  # Natural aging\n",
    "    age_indicators_fake = np.random.normal(0.3, 0.25, n_samples - n_authentic)  # Artificial aging or too new\n",
    "    \n",
    "    # Canvas/material analysis\n",
    "    canvas_authenticity_auth = np.random.normal(0.85, 0.1, n_authentic)  # Period-appropriate materials\n",
    "    canvas_authenticity_fake = np.random.normal(0.45, 0.2, n_samples - n_authentic)  # Modern materials\n",
    "    \n",
    "    # Pigment analysis (spectroscopic features)\n",
    "    pigment_match_auth = np.random.normal(0.9, 0.08, n_authentic)  # Period-correct pigments\n",
    "    pigment_match_fake = np.random.normal(0.5, 0.2, n_samples - n_authentic)  # Modern pigments\n",
    "    \n",
    "    # Style consistency (deep learning feature)\n",
    "    style_match_auth = np.random.normal(0.88, 0.1, n_authentic)  # Matches artist's style\n",
    "    style_match_fake = np.random.normal(0.4, 0.18, n_samples - n_authentic)  # Style mismatch\n",
    "    \n",
    "    # Provenance score (historical documentation)\n",
    "    provenance_auth = np.random.normal(0.75, 0.15, n_authentic)  # Good documentation\n",
    "    provenance_fake = np.random.normal(0.2, 0.15, n_samples - n_authentic)  # Poor/missing documentation\n",
    "    \n",
    "    # Combine features\n",
    "    features_authentic = np.column_stack([\n",
    "        color_variance_auth, texture_complexity_auth, brush_consistency_auth,\n",
    "        age_indicators_auth, canvas_authenticity_auth, pigment_match_auth,\n",
    "        style_match_auth, provenance_auth\n",
    "    ])\n",
    "    \n",
    "    features_fake = np.column_stack([\n",
    "        color_variance_fake, texture_complexity_fake, brush_consistency_fake,\n",
    "        age_indicators_fake, canvas_authenticity_fake, pigment_match_fake,\n",
    "        style_match_fake, provenance_fake\n",
    "    ])\n",
    "    \n",
    "    # Labels\n",
    "    labels_authentic = np.ones(n_authentic)\n",
    "    labels_fake = np.zeros(n_samples - n_authentic)\n",
    "    \n",
    "    # Combine and shuffle\n",
    "    X = np.vstack([features_authentic, features_fake])\n",
    "    y = np.hstack([labels_authentic, labels_fake])\n",
    "    \n",
    "    # Shuffle\n",
    "    indices = np.random.permutation(len(X))\n",
    "    X, y = X[indices], y[indices]\n",
    "    \n",
    "    # Create DataFrame\n",
    "    feature_names = [\n",
    "        'color_variance', 'texture_complexity', 'brush_consistency',\n",
    "        'age_indicators', 'canvas_authenticity', 'pigment_match',\n",
    "        'style_match', 'provenance_score'\n",
    "    ]\n",
    "    \n",
    "    df = pd.DataFrame(X, columns=feature_names)\n",
    "    df['is_authentic'] = y\n",
    "    \n",
    "    return df\n",
    "\n",
    "# Generate dataset\n",
    "artwork_data = generate_artwork_features(5000)\n",
    "\n",
    "print(f\"📊 Dataset generated: {len(artwork_data)} samples\")\n",
    "print(f\"🎨 Authentic artworks: {artwork_data['is_authentic'].sum():.0f} ({artwork_data['is_authentic'].mean()*100:.1f}%)\")\n",
    "print(f\"🔍 Suspected fakes: {(1-artwork_data['is_authentic']).sum():.0f} ({(1-artwork_data['is_authentic']).mean()*100:.1f}%)\")\n",
    "\n",
    "# Display sample\n",
    "display(artwork_data.head(10))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📈 Exploratory Data Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Feature distributions by authenticity\n",
    "fig, axes = plt.subplots(2, 4, figsize=(20, 10))\n",
    "axes = axes.ravel()\n",
    "\n",
    "feature_cols = [col for col in artwork_data.columns if col != 'is_authentic']\n",
    "\n",
    "for i, feature in enumerate(feature_cols):\n",
    "    authentic_data = artwork_data[artwork_data['is_authentic'] == 1][feature]\n",
    "    fake_data = artwork_data[artwork_data['is_authentic'] == 0][feature]\n",
    "    \n",
    "    axes[i].hist(authentic_data, alpha=0.7, label='Authentic', bins=30, color='green')\n",
    "    axes[i].hist(fake_data, alpha=0.7, label='Fake', bins=30, color='red')\n",
    "    axes[i].set_title(f'{feature.replace(\"_\", \" \").title()}')\n",
    "    axes[i].legend()\n",
    "    axes[i].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.suptitle('🎨 Feature Distributions: Authentic vs Fake Artworks', y=1.02, fontsize=16)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Correlation matrix\n",
    "plt.figure(figsize=(12, 8))\n",
    "correlation_matrix = artwork_data.corr()\n",
    "sns.heatmap(correlation_matrix, annot=True, cmap='RdYlBu_r', center=0, \n",
    "            square=True, linewidths=0.5)\n",
    "plt.title('🔍 Feature Correlation Matrix')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Statistical summary\n",
    "print(\"📊 Statistical Summary by Authenticity:\")\n",
    "print(\"\\n🎨 AUTHENTIC ARTWORKS:\")\n",
    "print(artwork_data[artwork_data['is_authentic'] == 1].describe())\n",
    "print(\"\\n🔍 SUSPECTED FAKES:\")\n",
    "print(artwork_data[artwork_data['is_authentic'] == 0].describe())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🤖 Model Development"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Prepare data for modeling\n",
    "X = artwork_data.drop('is_authentic', axis=1)\n",
    "y = artwork_data['is_authentic']\n",
    "\n",
    "# Train-test split\n",
    "X_train, X_test, y_train, y_test = train_test_split(\n",
    "    X, y, test_size=0.2, random_state=42, stratify=y\n",
    ")\n",
    "\n",
    "# Feature scaling\n",
    "scaler = StandardScaler()\n",
    "X_train_scaled = scaler.fit_transform(X_train)\n",
    "X_test_scaled = scaler.transform(X_test)\n",
    "\n",
    "print(f\"🎯 Training set: {len(X_train)} samples\")\n",
    "print(f\"🧪 Test set: {len(X_test)} samples\")\n",
    "print(f\"📊 Feature dimensions: {X_train.shape[1]}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize models\n",
    "models = {\n",
    "    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),\n",
    "    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),\n",
    "    'Logistic Regression': LogisticRegression(random_state=42),\n",
    "    'SVM': SVC(kernel='rbf', probability=True, random_state=42)\n",
    "}\n",
    "\n",
    "# Train and evaluate models\n",
    "model_results = {}\n",
    "\n",
    "for name, model in models.items():\n",
    "    print(f\"\\n🤖 Training {name}...\")\n",
    "    \n",
    "    # Use scaled data for SVM and Logistic Regression\n",
    "    if name in ['SVM', 'Logistic Regression']:\n",
    "        model.fit(X_train_scaled, y_train)\n",
    "        y_pred = model.predict(X_test_scaled)\n",
    "        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]\n",
    "    else:\n",
    "        model.fit(X_train, y_train)\n",
    "        y_pred = model.predict(X_test)\n",
    "        y_pred_proba = model.predict_proba(X_test)[:, 1]\n",
    "    \n",
    "    # Calculate metrics\n",
    "    accuracy = accuracy_score(y_test, y_pred)\n",
    "    precision = precision_score(y_test, y_pred)\n",
    "    recall = recall_score(y_test, y_pred)\n",
    "    f1 = f1_score(y_test, y_pred)\n",
    "    auc = roc_auc_score(y_test, y_pred_proba)\n",
    "    \n",
    "    model_results[name] = {\n",
    "        'model': model,\n",
    "        'accuracy': accuracy,\n",
    "        'precision': precision,\n",
    "        'recall': recall,\n",
    "        'f1': f1,\n",
    "        'auc': auc,\n",
    "        'predictions': y_pred,\n",
    "        'probabilities': y_pred_proba\n",
    "    }\n",
    "    \n",
    "    print(f\"✅ Accuracy: {accuracy:.3f}\")\n",
    "    print(f\"📊 Precision: {precision:.3f}\")\n",
    "    print(f\"🎯 Recall: {recall:.3f}\")\n",
    "    print(f\"⚖️ F1-Score: {f1:.3f}\")\n",
    "    print(f\"📈 AUC: {auc:.3f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Model Comparison and Evaluation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create comparison DataFrame\n",
    "comparison_data = []\n",
    "for name, results in model_results.items():\n",
    "    comparison_data.append({\n",
    "        'Model': name,\n",
    "        'Accuracy': results['accuracy'],\n",
    "        'Precision': results['precision'],\n",
    "        'Recall': results['recall'],\n",
    "        'F1-Score': results['f1'],\n",
    "        'AUC': results['auc']\n",
    "    })\n",
    "\n",
    "comparison_df = pd.DataFrame(comparison_data)\n",
    "comparison_df = comparison_df.sort_values('AUC', ascending=False)\n",
    "\n",
    "print(\"🏆 MODEL PERFORMANCE COMPARISON:\")\n",
    "print(\"=\" * 50)\n",
    "display(comparison_df.round(3))\n",
    "\n",
    "# Visualize model comparison\n",
    "fig, axes = plt.subplots(1, 2, figsize=(15, 6))\n",
    "\n",
    "# Bar plot of metrics\n",
    "metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']\n",
    "x = np.arange(len(comparison_df))\n",
    "width = 0.15\n",
    "\n",
    "for i, metric in enumerate(metrics_to_plot):\n",
    "    axes[0].bar(x + i*width, comparison_df[metric], width, label=metric, alpha=0.8)\n",
    "\n",
    "axes[0].set_xlabel('Models')\n",
    "axes[0].set_ylabel('Score')\n",
    "axes[0].set_title('🏆 Model Performance Comparison')\n",
    "axes[0].set_xticks(x + width * 2)\n",
    "axes[0].set_xticklabels(comparison_df['Model'], rotation=45)\n",
    "axes[0].legend()\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "# Best model confusion matrix\n",
    "best_model_name = comparison_df.iloc[0]['Model']\n",
    "best_predictions = model_results[best_model_name]['predictions']\n",
    "\n",
    "cm = confusion_matrix(y_test, best_predictions)\n",
    "sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1])\n",
    "axes[1].set_title(f'🎯 Confusion Matrix - {best_model_name}')\n",
    "axes[1].set_xlabel('Predicted')\n",
    "axes[1].set_ylabel('Actual')\n",
    "axes[1].set_xticklabels(['Fake', 'Authentic'])\n",
    "axes[1].set_yticklabels(['Fake', 'Authentic'])\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(f\"\\n🥇 BEST MODEL: {best_model_name}\")\n",
    "print(f\"📊 AUC Score: {comparison_df.iloc[0]['AUC']:.3f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🧠 Deep Learning Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Build neural network for authenticity detection\n",
    "def create_authenticity_model(input_dim):\n",
    "    \"\"\"\n",
    "    Create a neural network for artwork authenticity detection\n",
    "    \"\"\"\n",
    "    model = keras.Sequential([\n",
    "        layers.Dense(128, activation='relu', input_shape=(input_dim,)),\n",
    "        layers.BatchNormalization(),\n",
    "        layers.Dropout(0.3),\n",
    "        \n",
    "        layers.Dense(64, activation='relu'),\n",
    "        layers.BatchNormalization(),\n",
    "        layers.Dropout(0.3),\n",
    "        \n",
    "        layers.Dense(32, activation='relu'),\n",
    "        layers.Dropout(0.2),\n",
    "        \n",
    "        layers.Dense(16, activation='relu'),\n",
    "        layers.Dense(1, activation='sigmoid')  # Binary classification\n",
    "    ])\n",
    "    \n",
    "    model.compile(\n",
    "        optimizer=keras.optimizers.Adam(learning_rate=0.001),\n",
    "        loss='binary_crossentropy',\n",
    "        metrics=['accuracy', 'precision', 'recall']\n",
    "    )\n",
    "    \n",
    "    return model\n",
    "\n",
    "# Create and train neural network\n",
    "print(\"🧠 Building Deep Learning Model...\")\n",
    "nn_model = create_authenticity_model(X_train_scaled.shape[1])\n",
    "\n",
    "# Display model architecture\n",
    "print(\"\\n🏗️ Model Architecture:\")\n",
    "nn_model.summary()\n",
    "\n",
    "# Train model\n",
    "print(\"\\n🎯 Training Neural Network...\")\n",
    "history = nn_model.fit(\n",
    "    X_train_scaled, y_train,\n",
    "    epochs=50,\n",
    "    batch_size=32,\n",
    "    validation_split=0.2,\n",
    "    verbose=1,\n",
    "    callbacks=[\n",
    "        keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),\n",
    "        keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)\n",
    "    ]\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Evaluate neural network\n",
    "nn_predictions = nn_model.predict(X_test_scaled)\n",
    "nn_pred_binary = (nn_predictions > 0.5).astype(int).flatten()\n",
    "nn_pred_proba = nn_predictions.flatten()\n",
    "\n",
    "# Calculate metrics\n",
    "nn_accuracy = accuracy_score(y_test, nn_pred_binary)\n",
    "nn_precision = precision_score(y_test, nn_pred_binary)\n",
    "nn_recall = recall_score(y_test, nn_pred_binary)\n",
    "nn_f1 = f1_score(y_test, nn_pred_binary)\n",
    "nn_auc = roc_auc_score(y_test, nn_pred_proba)\n",
    "\n",
    "print(\"\\n🧠 NEURAL NETWORK PERFORMANCE:\")\n",
    "print(\"=\" * 40)\n",
    "print(f\"✅ Accuracy: {nn_accuracy:.3f}\")\n",
    "print(f\"📊 Precision: {nn_precision:.3f}\")\n",
    "print(f\"🎯 Recall: {nn_recall:.3f}\")\n",
    "print(f\"⚖️ F1-Score: {nn_f1:.3f}\")\n",
    "print(f\"📈 AUC: {nn_auc:.3f}\")\n",
    "\n",
    "# Plot training history\n",
    "fig, axes = plt.subplots(1, 2, figsize=(15, 5))\n",
    "\n",
    "# Loss\n",
    "axes[0].plot(history.history['loss'], label='Training Loss')\n",
    "axes[0].plot(history.history['val_loss'], label='Validation Loss')\n",
    "axes[0].set_title('📉 Model Loss')\n",
    "axes[0].set_xlabel('Epoch')\n",
    "axes[0].set_ylabel('Loss')\n",
    "axes[0].legend()\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "# Accuracy\n",
    "axes[1].plot(history.history['accuracy'], label='Training Accuracy')\n",
    "axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy')\n",
    "axes[1].set_title('📈 Model Accuracy')\n",
    "axes[1].set_xlabel('Epoch')\n",
    "axes[1].set_ylabel('Accuracy')\n",
    "axes[1].legend()\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎯 Feature Importance Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Feature importance from best traditional model\n",
    "best_traditional_model = model_results[best_model_name]['model']\n",
    "\n",
    "if hasattr(best_traditional_model, 'feature_importances_'):\n",
    "    feature_importance = best_traditional_model.feature_importances_\n",
    "    feature_names = X.columns\n",
    "    \n",
    "    # Create feature importance dataframe\n",
    "    importance_df = pd.DataFrame({\n",
    "        'feature': feature_names,\n",
    "        'importance': feature_importance\n",
    "    }).sort_values('importance', ascending=False)\n",
    "    \n",
    "    # Plot feature importance\n",
    "    plt.figure(figsize=(12, 8))\n",
    "    bars = plt.barh(importance_df['feature'], importance_df['importance'])\n",
    "    plt.xlabel('Feature Importance')\n",
    "    plt.title(f'🎯 Feature Importance - {best_model_name}')\n",
    "    plt.gca().invert_yaxis()\n",
    "    \n",
    "    # Color bars based on importance\n",
    "    colors = plt.cm.RdYlGn(importance_df['importance'] / importance_df['importance'].max())\n",
    "    for bar, color in zip(bars, colors):\n",
    "        bar.set_color(color)\n",
    "    \n",
    "    plt.grid(True, alpha=0.3)\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    print(\"🎯 FEATURE IMPORTANCE RANKING:\")\n",
    "    print(\"=\" * 35)\n",
    "    for i, (_, row) in enumerate(importance_df.iterrows(), 1):\n",
    "        print(f\"{i:2d}. {row['feature'].replace('_', ' ').title():<20} {row['importance']:.3f}\")\n",
    "else:\n",
    "    print(f\"Feature importance not available for {best_model_name}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔍 Model Interpretation and Examples"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create example predictions with explanations\n",
    "def explain_prediction(model, scaler, sample_features, feature_names, use_scaling=True):\n",
    "    \"\"\"\n",
    "    Explain a single prediction with feature contributions\n",
    "    \"\"\"\n",
    "    if use_scaling:\n",
    "        sample_scaled = scaler.transform([sample_features])\n",
    "        prediction = model.predict_proba(sample_scaled)[0][1]\n",
    "    else:\n",
    "        prediction = model.predict_proba([sample_features])[0][1]\n",
    "    \n",
    "    return prediction\n",
    "\n",
    "# Test examples\n",
    "test_examples = [\n",
    "    {\n",
    "        'name': 'Suspicious Modern Painting',\n",
    "        'features': [0.35, 4.2, 0.45, 0.1, 0.3, 0.2, 0.3, 0.1],\n",
    "        'description': 'High color variance, low texture complexity, poor provenance'\n",
    "    },\n",
    "    {\n",
    "        'name': 'Authentic Renaissance Work',\n",
    "        'features': [0.12, 7.8, 0.85, 0.8, 0.9, 0.95, 0.9, 0.8],\n",
    "        'description': 'Natural aging, consistent brush work, excellent provenance'\n",
    "    },\n",
    "    {\n",
    "        'name': 'Questionable Attribution',\n",
    "        'features': [0.2, 6.5, 0.7, 0.6, 0.7, 0.8, 0.5, 0.4],\n",
    "        'description': 'Mixed signals - some authentic features, poor style match'\n",
    "    }\n",
    "]\n",
    "\n",
    "print(\"🔍 EXAMPLE PREDICTIONS AND EXPLANATIONS:\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "for example in test_examples:\n",
    "    # Get predictions from best models\n",
    "    traditional_pred = explain_prediction(\n",
    "        best_traditional_model, scaler, example['features'], \n",
    "        X.columns, use_scaling=(best_model_name in ['SVM', 'Logistic Regression'])\n",
    "    )\n",
    "    \n",
    "    if best_model_name in ['SVM', 'Logistic Regression']:\n",
    "        sample_scaled = scaler.transform([example['features']])\n",
    "        nn_pred = nn_model.predict(sample_scaled)[0][0]\n",
    "    else:\n",
    "        sample_scaled = scaler.transform([example['features']])\n",
    "        nn_pred = nn_model.predict(sample_scaled)[0][0]\n",
    "    \n",
    "    print(f\"\\n🎨 {example['name']}\")\n",
    "    print(f\"📋 Description: {example['description']}\")\n",
    "    print(f\"🤖 {best_model_name} Authenticity Score: {traditional_pred:.3f} ({'✅ AUTHENTIC' if traditional_pred > 0.5 else '❌ SUSPICIOUS'})\")\n",
    "    print(f\"🧠 Neural Network Score: {nn_pred:.3f} ({'✅ AUTHENTIC' if nn_pred > 0.5 else '❌ SUSPICIOUS'})\")\n",
    "    \n",
    "    # Feature breakdown\n",
    "    print(\"📊 Feature Analysis:\")\n",
    "    for i, (feature, value) in enumerate(zip(X.columns, example['features'])):\n",
    "        status = \"✅\" if (\n",
    "            (feature in ['texture_complexity', 'brush_consistency', 'age_indicators', \n",
    "                        'canvas_authenticity', 'pigment_match', 'style_match', 'provenance_score'] and value > 0.6) or\n",
    "            (feature == 'color_variance' and value < 0.2)\n",
    "        ) else \"⚠️\"\n",
    "        print(f\"  {status} {feature.replace('_', ' ').title()}: {value:.2f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 💾 Model Deployment Preparation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save the best models\n",
    "import joblib\n",
    "import os\n",
    "\n",
    "# Create models directory\n",
    "os.makedirs('../models', exist_ok=True)\n",
    "\n",
    "# Save traditional model\n",
    "joblib.dump(best_traditional_model, f'../models/best_traditional_model_{best_model_name.lower().replace(\" \", \"_\")}.pkl')\n",
    "joblib.dump(scaler, '../models/feature_scaler.pkl')\n",
    "\n",
    "# Save neural network\n",
    "nn_model.save('../models/authenticity_neural_network.h5')\n",
    "\n",
    "# Save feature names\n",
    "feature_info = {\n",
    "    'feature_names': list(X.columns),\n",
    "    'feature_descriptions': {\n",
    "        'color_variance': 'Variation in color distribution (lower = more natural)',\n",
    "        'texture_complexity': 'Complexity of surface texture patterns',\n",
    "        'brush_consistency': 'Consistency of brush stroke patterns',\n",
    "        'age_indicators': 'Presence of natural aging signs',\n",
    "        'canvas_authenticity': 'Authenticity of canvas/material',\n",
    "        'pigment_match': 'Match with period-appropriate pigments',\n",
    "        'style_match': 'Consistency with artist\\'s known style',\n",
    "        'provenance_score': 'Quality of historical documentation'\n",
    "    },\n",
    "    'model_performance': {\n",
    "        'best_traditional_model': best_model_name,\n",
    "        'traditional_auc': comparison_df.iloc[0]['AUC'],\n",
    "        'neural_network_auc': nn_auc,\n",
    "        'recommendation': 'Use ensemble of both models for critical decisions'\n",
    "    }\n",
    "}\n",
    "\n",
    "import json\n",
    "with open('../models/model_info.json', 'w') as f:\n",
    "    json.dump(feature_info, f, indent=2)\n",
    "\n",
    "print(\"💾 MODELS SAVED SUCCESSFULLY!\")\n",
    "print(\"=\" * 35)\n",
    "print(f\"📁 Best Traditional Model: {best_model_name}\")\n",
    "print(f\"🧠 Neural Network: Saved as authenticity_neural_network.h5\")\n",
    "print(f\"📊 Feature Scaler: Saved for preprocessing\")\n",
    "print(f\"📋 Model Info: Complete metadata saved\")\n",
    "\n",
    "print(\"\\n🚀 DEPLOYMENT READY!\")\n",
    "print(\"Models can be loaded and used for:\")\n",
    "print(\"- Real-time artwork authentication\")\n",
    "print(\"- Batch processing of museum collections\")\n",
    "print(\"- Integration with digital preservation systems\")\n",
    "print(\"- API services for art dealers and collectors\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📝 Model Performance Summary"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Final summary\n",
    "print(\"🎨 DIGITAL ALEXANDRIA - AUTHENTICITY DETECTION MODEL\")\n",
    "print(\"=\" * 60)\n",
    "